In [25]:
import networkx as nx
import requests as rq
from routingapp.common.network_stat_utils import get_inter_group_edges, link_with_port_mn_to_hmap

In [26]:
json_graph = rq.get('http://0.0.0.0:8000/graph').json()
graph = nx.json_graph.node_link_graph(json_graph)
inter_group_edges = get_inter_group_edges(graph)
inter_group_edges

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]

In [27]:
link_with_port = link_with_port_mn_to_hmap()
s1, s2 = inter_group_edges[0]
p1 = link_with_port[(f's{s1}', f's{s2}')]['port1']
p2 = link_with_port[(f's{s1}', f's{s2}')]['port2']

In [28]:
deltal_port_stat = rq.get('http://0.0.0.0:8080/delta_port_stat').json()
deltal_port_stat
# link_usage_s1 = ...
# link_usage_s2 = ...

[{'dpid': 6,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 6,
  'port_no': 2,
  'tx_packets': 21,
  'rx_packets': 18,
  'tx_bytes': 1944,
  'rx_bytes': 1764,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 6,
  'port_no': 3,
  'tx_packets': 11,
  'rx_packets': 14,
  'tx_bytes': 964,
  'rx_bytes': 1220,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 6,
  'port_no': 4,
  'tx_packets': 11,
  'rx_packets': 10,
  'tx_bytes': 926,
  'rx_bytes': 904,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 3000000},
 {'dpid': 5,
  'port_no': 2,
  'tx_packets': 29,
  'rx_packets': 30,
  'tx_bytes': 2728,
  'rx_bytes': 2940,
  'rx_error': 0,
  'd

In [29]:
dps_hmap = {}
for d in deltal_port_stat:
    key = (d['dpid'], d['port_no'])
    dps_hmap[key] = d
    
dps_hmap

{(6, 1): {'dpid': 6,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 2): {'dpid': 6,
  'port_no': 2,
  'tx_packets': 21,
  'rx_packets': 18,
  'tx_bytes': 1944,
  'rx_bytes': 1764,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 3): {'dpid': 6,
  'port_no': 3,
  'tx_packets': 11,
  'rx_packets': 14,
  'tx_bytes': 964,
  'rx_bytes': 1220,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 4): {'dpid': 6,
  'port_no': 4,
  'tx_packets': 11,
  'rx_packets': 10,
  'tx_bytes': 926,
  'rx_bytes': 904,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 1): {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 3000000},
 (5, 2): {'dpid': 5,
  'port_no': 2,
  'tx_packets': 29,
  'rx_packets': 30,
  'tx_bytes':

In [30]:
def get_controller_list():
  """Fetches controller list from the API and converts string/int keys to int keys.

  Returns:
    A list of dictionaries with integer keys.
  """
  ctrler_list = rq.get('http://0.0.0.0:8000/controller_list').json()

  # Use list comprehension for concise conversion
  new_ctrler_list = [{int(key): value} for ctrler in ctrler_list for key, value in ctrler.items() if key.isdigit()]

  return new_ctrler_list

get_controller_list()


[{0: {'ip': '0.0.0.0', 'port': 6633}}, {1: {'ip': '0.0.0.0', 'port': 6634}}]

In [31]:
test = rq.get('http://0.0.0.0:8000/controller_list').json()
type(test)

list

In [32]:
from routingapp.common.network_stat_utils import get_sw_ctrler_mapping
RYU_PORT = 8080
def get_all_delta_port_stat():
    ''' Get delta port stat of all controllers in the network
    Return:
    hashmap of (dpid, port):{delta_port_stat}
    '''
    deltal_port_stat = []
    ctrler_list = get_controller_list()
    
    for ctrler in ctrler_list:
        for key, value in ctrler.items():
            deltal_port_stat += rq.get(f'http://{value.get("ip")}:{RYU_PORT+key}/delta_port_stat').json()

    for d in deltal_port_stat:
        key = (d['dpid'], d['port_no'])
        dps_hmap[key] = d

    return dps_hmap

get_all_delta_port_stat()

{(6, 1): {'dpid': 6,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 2): {'dpid': 6,
  'port_no': 2,
  'tx_packets': 21,
  'rx_packets': 18,
  'tx_bytes': 1944,
  'rx_bytes': 1764,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 3): {'dpid': 6,
  'port_no': 3,
  'tx_packets': 11,
  'rx_packets': 14,
  'tx_bytes': 964,
  'rx_bytes': 1220,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 4): {'dpid': 6,
  'port_no': 4,
  'tx_packets': 11,
  'rx_packets': 10,
  'tx_bytes': 926,
  'rx_bytes': 904,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 1): {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 3000000},
 (5, 2): {'dpid': 5,
  'port_no': 2,
  'tx_packets': 29,
  'rx_packets': 30,
  'tx_bytes':

In [33]:
get_inter_group_edges(graph)

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]

In [37]:
inter_group_edges = get_inter_group_edges(graph)
deltal_port_stat = get_all_delta_port_stat()
link_with_port = link_with_port_mn_to_hmap()

ige = inter_group_edges[0]

link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port1']
link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port2']

3

In [75]:
import multiprocessing
RYU_PORT = 8080
RESTHOOKMN_PORT = 8000
def get_link_quality_multi_controller():
    num_ctrler = rq.get(f'http://0.0.0.0:{RESTHOOKMN_PORT}/controller_list').json()
    lqcs = []
    for i in range(len(num_ctrler)):
            lqcs += rq.get(f'http://0.0.0.0:{RYU_PORT}/link_quality').json()
    return lqcs
get_link_quality_multi_controller()

[{'src.dpid': 1,
  'dst.dpid': 3,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 1,
  'dst.dpid': 5,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 1,
  'dst.dpid': 2,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 6,
  'dst.dpid': 5,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0.0027884038653784404,
  'free_bandwidth': None},
 {'src.dpid': 6,
  'dst.dpid': 2,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0.0032095968010665797,
  'free_bandwidth': None},
 {'src.dpid': 4,
  'dst.dpid': 3,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 4,
  'dst.dpid': 5,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 2,
  'dst.dpid': 1,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},


In [77]:
link_with_port_mn_to_hmap()

{('s1', 'h1'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 'h1',
  'port1': 1,
  'port2': 0},
 ('h1', 's1'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 'h1',
  'node2': 's1',
  'port1': 0,
  'port2': 1},
 ('s1', 'h1000'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 'h1000',
  'port1': 2,
  'port2': 0},
 ('h1000', 's1'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 'h1000',
  'node2': 's1',
  'port1': 0,
  'port2': 2},
 ('s1', 's2'): {'bw': 141,
  'delay': '35ms',
  'loss': 3,
  'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 's2',
  'port1': 3,
  'port2': 3},
 ('s2', 's1'): {'bw': 141,
  'delay': '35ms',
  'loss': 3,
  'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's2',
  'node2': 's1',
  'port1': 3,
  'port2': 3},
 ('s1', 's3'): {'bw': 105,
  'delay': '14ms',
  'loss': 2,
  'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 's3',
  'port1': 4,
  'port2': 3},

In [71]:
def get_inter_group_edges_link_traffic():
    '''
        
    '''
    inter_group_edges = get_inter_group_edges(graph)
    deltal_port_stat = get_all_delta_port_stat()
    link_with_port = link_with_port_mn_to_hmap()
    
    inter_port_stat = []
    print(inter_group_edges)
    for ige in inter_group_edges:
        node1 = ige[0]
        port1 = link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port1']
        node2 = ige[1]
        port2 = link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port2']

        node1_port_stat = deltal_port_stat.get((node1, port1))
        node2_port_stat = deltal_port_stat.get((node2, port2))
        
        # ref:
        '''
            bandwidth = min(src_free_bandwidth, dst_free_bandwidth)
            link_usage = min(src_link_usage, dst_link_usage)
        '''
        # link usage = delta upload+download min(src_dpid_port, dst_dpid_port)
        # min()

        node1_traffic = node1_port_stat['tx_bytes'] + node1_port_stat['rx_bytes']
        node2_traffic = node2_port_stat['tx_bytes'] + node2_port_stat['rx_bytes']
        link_traffic = (min(node1_traffic, node2_traffic)) / (8*1000000)
        inter_port_stat.append({
            'src.dpid': node1,
            'dst.dpid': node2,
            'link_usage': link_traffic,
        })
    return inter_port_stat
get_inter_group_edges_link_traffic()

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]


[{'src.dpid': 1, 'dst.dpid': 2, 'link_usage': 0.00010625},
 {'src.dpid': 1, 'dst.dpid': 5, 'link_usage': 9.4e-05},
 {'src.dpid': 2, 'dst.dpid': 1, 'link_usage': 0.00010625},
 {'src.dpid': 5, 'dst.dpid': 1, 'link_usage': 9.4e-05},
 {'src.dpid': 5, 'dst.dpid': 4, 'link_usage': 6.95e-05},
 {'src.dpid': 4, 'dst.dpid': 5, 'link_usage': 6.95e-05}]

[{'src.dpid': 1,
  'dst.dpid': 3,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 1,
  'dst.dpid': 5,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 1,
  'dst.dpid': 2,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 6,
  'dst.dpid': 5,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0.003470843052328282,
  'free_bandwidth': None},
 {'src.dpid': 6,
  'dst.dpid': 2,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0.0026871042985767346,
  'free_bandwidth': None},
 {'src.dpid': 4,
  'dst.dpid': 3,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 4,
  'dst.dpid': 5,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 2,
  'dst.dpid': 1,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 

In [73]:
def get_multi_domain_link_info():
    
    inter_group_edges = get_inter_group_edges(graph)
    deltal_port_stat = get_all_delta_port_stat()
    link_with_port = link_with_port_mn_to_hmap()
    
    # Only get link_usage
    link_quality_multi_controller = get_link_quality_multi_controller()
    # get packetloss+delay directly from mininet
    link_quality_mininet = 
    link_ping_stat = rq.get('http://0.0.0.0:8000/link_ping_stat').json()
            
    lqc_hmap = {}
    lqm_hmap = {}
    lps_hmap = {}
    
    for d in link_quality_multi_controller:
        key = (d['src.dpid'], d['dst.dpid'])
        lqc_hmap[key] = d

    for d in link_quality_mininet:
        key = (d['src.dpid'], d['dst.dpid'])
        lqm_hmap[key] = d
    
    for d in link_ping_stat:
        key = (d['src.host'], d['dst.host'])
        lps_hmap[key] = d

    link_quality = []
    for key, lqc_value in lqc_hmap.items():
        try: 
            lqm_value = lqm_hmap.get(key)
            lps_value = lps_hmap.get(key)
            if lqm_value == None: continue  
            link_quality.append({
                'src.dpid': key[0],
                'dst.dpid': key[1],
                'packet_loss': lps_value.get('packet_loss', None),
                'delay': lps_value.get('delay', None),
                'bandwidth': lqm_value.get('bandwidth', 1),
                'link_usage': lqc_value.get('link_usage', 0),
                'link_utilization': lqc_value.get('link_usage', 0) / lqm_value.get('bandwidth', 1) * 100,
            })
        except:
            ...
        
    return link_quality
        
get_multi_domain_link_info()
   

KeyError: 'src.dpid'